# **Obtaining Data with Python for Beta Estimation and Portfolio Optimization**
This tutorial guides you through **downloading, inspecting, and processing financial data** to perform **beta estimation and portfolio optimization**. Each step includes code explanations to help you master these techniques.

## **Overview of Python Libraries for Financial Analysis**
1. **`pandas`**: Used for data manipulation and analysis.
2. **`yfinance`**: Fetches stock price data from Yahoo Finance.
3. **`openpyxl`**: Reads and writes Excel files.
4. **`statsmodels`**: Performs statistical analysis like regression.
5. **`matplotlib`**: Plots data for insights and trends.

## **Installing Required Libraries**

In [ ]:
# Uncomment and run this code to install necessary libraries
# !pip3 install pandas yfinance openpyxl matplotlib statsmodels seaborn pyarrow PyPortfolioOpt

## **Step 1: Downloading Data Using Bash Commands**

In [ ]:
!mkdir financial_data
!curl -o financial_data/ff_data.csv http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors.CSV
!curl -o financial_data/erp_data.xlsx https://pages.stern.nyu.edu/~adamodar/pc/ERPbymonth.xlsx

#### Code Explanation:
- **`mkdir`**: Creates a new directory for storing the data.
- **`curl`**: Downloads the CSV and Excel datasets from their respective URLs.

## **Step 2: Inspecting the Files**

In [ ]:
!ls -lh financial_data
!head -n 10 financial_data/ff_data.csv
!tail -n 10 financial_data/ff_data.csv
!file financial_data/erp_data.xlsx

#### Code Explanation:
- **`ls -lh`**: Lists the contents of the directory with readable sizes.
- **`head`** and **`tail`**: Display the first and last few lines of the CSV file.
- **`file`**: Confirms the format of the downloaded Excel file.


## **Step 3a: Loading Data into Python**

In [ ]:
import pandas as pd

# Load the Fama/French data
fama_french = pd.read_csv('financial_data/ff_data.csv', skiprows=3, skipfooter=1, engine='python')
fama_french.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
fama_french = fama_french.dropna()[fama_french.dropna().Date.map(lambda x: int(x)>9999)]
fama_french['Date'] = pd.to_datetime(fama_french['Date'].astype(str), format='%Y%m').dt.strftime('%Y-%m')
fama_french = fama_french[['Date', 'Mkt-RF', 'RF']]
latest_60_months = fama_french.iloc[-60:].copy()
latest_60_months[['Mkt-RF', 'RF']] = latest_60_months[['Mkt-RF','RF']].apply(pd.to_numeric, errors='coerce', axis=1)
latest_60_months.info()

### **Code Explanation:**

1. **`import pandas as pd`**  
   - Imports the `pandas` library for data manipulation and analysis.

2. **`pd.read_csv()`**  
   - Loads the **Fama-French dataset** from a CSV file.  
   - **`skiprows=3`**: Skips the first three rows (usually headers or metadata).  
   - **`skipfooter=1`**: Skips the last row (likely a summary or footer).  
   - **`engine='python'`**: Uses the Python engine for parsing.

3. **`rename()`**  
   - Renames the first column from `Unnamed: 0` to **`Date`** for clarity.  
   - **`inplace=True`**: Modifies the DataFrame in place.

4. **`dropna()` and `Date.map()`**  
   - Removes rows with missing values.  
   - **`Date.map(lambda x: int(x) > 9999)`**: Filters rows to keep only those with valid **year-month** dates.

5. **`pd.to_datetime()` and `strftime()`**  
   - Converts the **`Date`** column to datetime using the **`%Y%m`** format.  
   - **`strftime('%Y-%m')`**: Formats the dates as `YYYY-MM` strings for consistency.

6. **Column Selection**  
   - Selects the **`Date`**, **`Mkt-RF`** (market excess return), and **`RF`** (risk-free rate) columns for further analysis.

7. **`iloc[-60:]` and `copy()`**  
   - Extracts the **last 60 rows** (the latest 60 months of data) and makes a copy to avoid altering the original DataFrame.

8. **`apply(pd.to_numeric, errors='coerce', axis=1)`**  
   - Converts the **`Mkt-RF`** and **`RF`** columns to numeric values.  
   - **`errors='coerce'`**: Replaces any non-numeric values with NaN.

9. **`info()`**  
   - Displays a **summary of the DataFrame**, including the data types of each column and non-null value counts, to ensure everything is correctly formatted.

## **Step 3b: Download Historical Stock Prices Using Yahoo Finance**

In [ ]:
import yfinance as yf
# Updated list of 20 tickers (replacing TWTR with SHOP)
ticker_list = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA', 'INTC', 'ORCL',
               'IBM', 'ADBE', 'CSCO', 'QCOM', 'AMD', 'SAP', 'PYPL', 'CRM', 'UBER', 'SHOP']

# Define the date range
start_date = '2017-08-01'
end_date = '2022-08-31'

# Download data for multiple tickers
data = yf.download(
    tickers=ticker_list,
    start=start_date,
    end=end_date,
    interval='1mo',
    group_by='ticker'
)

# Extract 'Adj Close' for each ticker and rename to 'Adjusted Price'
adjusted_data = pd.DataFrame()

for ticker in ticker_list:
    # Select and rename the 'Adj Close' column
    ticker_data = data[ticker][['Adj Close']].rename(columns={'Adj Close': 'Adjusted Price'})
    # Add the ticker symbol as a column
    ticker_data['Ticker'] = ticker
    # Append the data to the main DataFrame
    adjusted_data = pd.concat([adjusted_data, ticker_data])

# Reset index to make it more readable
adjusted_data.reset_index(inplace=True)

# Transpose the data using pivot_table
transposed_data = adjusted_data.pivot_table(values='Adjusted Price', index='Date', columns='Ticker')


### **Code Explanation:**

1. **`import yfinance as yf`**  
   - Imports the `yfinance` library for fetching stock market data.

2. **`ticker_list`**  
   - A list containing 20 stock tickers, including companies like **Apple (AAPL)**, **Microsoft (MSFT)**, and **Shopify (SHOP)**. These tickers represent the stocks for which data will be retrieved.

3. **`start_date` and `end_date`**  
   - Define the time period for the stock data:  
     - **Start Date:** August 1, 2017.  
     - **End Date:** August 31, 2022.  
   - This range ensures the download includes 5 years of monthly stock data.

4. **`yf.download()`**  
   - Downloads historical stock data for the tickers in the list.  
   - **`tickers=ticker_list`**: Retrieves data for all 20 tickers.  
   - **`interval='1mo'`**: Fetches monthly data.  
   - **`group_by='ticker'`**: Organizes the data so each ticker's data is kept separate within the same DataFrame.

5. **Creating an Empty DataFrame**  
   - Initializes an empty DataFrame called `adjusted_data` to store the cleaned data for all tickers.

6. **Looping Through Tickers**  
   - For each ticker in the list:
     - Extracts the **Adjusted Close** price to account for dividends and stock splits.
     - Renames the **'Adj Close'** column to **'Adjusted Price'** for better clarity.
     - Adds a new column to store the ticker symbol for reference.

7. **Appending Data with `pd.concat()`**  
   - Each ticker’s data is appended to the main DataFrame, combining all the stock data into a single structured DataFrame.

8. **Resetting the Index**  
   - Resets the index to convert the date from the index back to a column for easier manipulation and display.

9. **Using `pivot_table()`**  
   - Transposes the DataFrame, organizing it with **dates as rows** and **tickers as columns**.  
   - This transformation provides a clean view, where each column shows the adjusted prices for a specific ticker over time. 

This final structure makes it easy to analyze the performance of multiple stocks simultaneously, with dates aligned for comparison across the different companies.

## **Step 4: Calculating Stock Returns with `pct_change()`**

In [ ]:
returns_data = transposed_data.pct_change() * 100
returns_data = returns_data.rename(columns=lambda x: x + '_return')
returns_data = returns_data.dropna()
returns_data = returns_data.reset_index()
returns_data['Date'] = pd.to_datetime(returns_data['Date'].astype(str), format='%Y-%m-%d %H:%M:%S%z').dt.strftime('%Y-%m')

### **Code Explanation:**

1. **`transposed_data.pct_change()`**  
   - Calculates the **percentage change** between consecutive rows (monthly returns) for each ticker.  
   - The result shows how much each stock’s price has changed (in percentage) from one month to the next.

2. **`* 100`**  
   - Converts the fractional percentage change values into **actual percentage values** by multiplying by 100.

3. **`rename(columns=lambda x: x + '_return')`**  
   - Renames each column by appending **`_return`** to the original ticker symbol (e.g., `AAPL` becomes `AAPL_return`).  
   - This clearly distinguishes these columns as **return values** rather than raw prices.

4. **`dropna()`**  
   - Removes any **rows with missing values** (NaN). This typically happens if there are no previous data points to compute percentage changes, such as the first row for each ticker.

5. **`reset_index()`**  
   - Resets the DataFrame’s index to make the data easier to read and manipulate.  
   - The original date index becomes a regular column.

6. **`pd.to_datetime()`**  
   - Converts the **'Date'** column into a proper datetime object to ensure consistent date formatting.

7. **`astype(str)`**  
   - Converts the date values into **string format**, so they can be formatted further.

8. **`strftime('%Y-%m')`**  
   - Formats the **Date** column to show only the **year and month** (e.g., `2022-08`), which aligns with the monthly intervals of the data.

This code processes the transposed stock data to calculate and format monthly returns for each stock, ensuring that the data is clean, structured, and ready for further analysis or visualization.